# Introduction to NLP Fundamentals in TensorFlow

Natural Language Processing (NLP) is a field in machine learning with the ability of a computer to understand, interpret, and generate human language. NLP is a subset of artificial intelligence and is widely used in applications like language translation, chatbots, sentiment analysis, speech recognition, etc.

## Chech for GPU

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1650 (UUID: GPU-8f599f8c-acce-243f-ba7d-fec42d1f06e9)


## Get Helper Functions

In [2]:
### We create a bunch of helpful functions throughout the course.
### Storing them here so they're easily accessible.

import tensorflow as tf

# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img

# Note: The following confusion matrix code is a remix of Scikit-Learn's 
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False): 
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).
  
  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """  
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")
  
# Make a function to predict on images and plot them (works with multi-class)
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);
  
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.

  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"

  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

# Plot the validation and training data separately
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

def compare_historys(original_history, new_history, initial_epochs=5):
    """
    Compares two TensorFlow model History objects.
    
    Args:
      original_history: History object from original model (before new_history)
      new_history: History object from continued model training (after original_history)
      initial_epochs: Number of epochs in original_history (new_history plot starts from here) 
    """
    
    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()
  
# Create function to unzip a zipfile into current working directory 
# (since we're going to be downloading and unzipping a few files)
import zipfile

def unzip_data(filename):
  """
  Unzips filename into the current working directory.

  Args:
    filename (str): a filepath to a target zip folder to be unzipped.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

# Walk through an image classification directory and find out how many files (images)
# are in each subdirectory.
import os

def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.

  Args:
    dir_path (str): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")
    
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
      y_true: true labels in the form of a 1D array
      y_pred: predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


## Get Data

In [3]:
import pandas as pd
train_df = pd.read_csv("data/train/train.csv")
test_df = pd.read_csv("data/test/test.csv")

## Visualizing Data

In [4]:
train_df.head(), test_df.head()

(   id keyword location                                               text  \
 0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
 1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
 2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
 3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
 4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   
 
    target  
 0       1  
 1       1  
 2       1  
 3       1  
 4       1  ,
    id keyword location                                               text
 0   0     NaN      NaN                 Just happened a terrible car crash
 1   2     NaN      NaN  Heard about #earthquake is different cities, s...
 2   3     NaN      NaN  there is a forest fire at spot pond, geese are...
 3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
 4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan)

In [5]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [6]:
# Shuffle the training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# Test data frame looks like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# Examples of each class
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# Total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Visualize the random training samples
import random
import matplotlib.pyplot as plt

random_index = random.randint(0, len(train_df)-5) # create random indexes not exceeding total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not real disaster)
Text:
Alhaji Putin is far from being a good person sha. At least I had front row seat to his complete obliteration of Ibeto cement a competitor.

---

Target: 0 (not real disaster)
Text:
A random hole just broke out in the street ?? http://t.co/dWU8QqYs0v

---

Target: 0 (not real disaster)
Text:
Help yourself or those you love who suffer from self-esteem wounds. You can today! http://t.co/tu6ScRSXVG http://t.co/iDhj4JBQ05

---

Target: 1 (real disaster)
Text:
U.S. Forest Service firefighter David Ruhl 38 died in the 'Frog Fire' in the Modoc National Forest. He had been temporarily reassigned

---

Target: 0 (not real disaster)
Text:
http://t.co/ETkd58Un8n - Cleveland Heights Shaker Heights fight blight: The House Next Door http://t.co/LRelVrm06w

---



### Split data into training and test sets

In [11]:
# Split the training data into training and validation sets
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

First step in working with text data is to convert them into numbers. There are few ways to do this:
* Tokenization - Direct mapping of token (a token could be a word or a character) to number.
* Embedding - Create a matrix of feature vector for each token (word) and use those vectors as features for the model.

### Text Vectorization (Tokenization)

In [14]:
train_sentences[:5], train_labels[:5]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
       dtype=object),
 array([0, 0, 1, 0, 0], dtype=int64))

In [15]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization variables
text_vectorization = TextVectorization(max_tokens=10000, # Set max_tokens to a specific integer value
                                       standardize="lower_and_strip_punctuation",
                                       split="whitespace",
                                       ngrams=None, # Create groups of n-words?
                                       output_mode="int", # How to map tokens to numbers
                                       output_sequence_length=None, # How long do you want your sequences to be
                                       pad_to_max_tokens=True)

In [16]:
len(train_sentences[0].split())

7

In [17]:
# Find the average number of tokens in the training data
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [18]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length of the sequence

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [19]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [20]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in the streets of New York"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 264,    3,  232,    4,    2, 1585,    6,   50,  913,    0,    0,
           0,    0,    0,    0]], dtype=int64)>

In [21]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Budget? Oh I am in trouble... but yes I would agree. #VarageSale  @Candace_Dx      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[2175,  392,    8,  160,    4,  495,   30,  635,    8,   78, 1331,
        7073,    1,    0,    0]], dtype=int64)>

In [22]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") # most common words
print(f"Bottom 5 least common words: {bottom_5_words}") # least common words

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [23]:
# Length of the vocabulary
len(words_in_vocab)

10000

## Creating an Embedding using an Embedding Layer

In [24]:
max_length

15

In [25]:
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                                      output_dim=128, # set size of embedding vector
                                      embeddings_initializer="uniform", # default initilization
                                      input_length=max_length) # how long is each input
embedding

In [26]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
ALERT! Sandy Hook Elementary School Evacuated After Û÷Bomb ThreatÛª http://t.co/LwLexXjUS8      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03153504,  0.00619791, -0.00067149, ..., -0.02033448,
          0.0397325 ,  0.02968505],
        [ 0.03170076, -0.03528668,  0.04349022, ...,  0.02769377,
          0.03216417, -0.04969713],
        [ 0.04305191,  0.01054632, -0.03586094, ...,  0.00923597,
         -0.04936764,  0.00768653],
        ...,
        [ 0.00946743,  0.00878171, -0.03614209, ..., -0.02412263,
          0.03870462,  0.01789257],
        [ 0.00946743,  0.00878171, -0.03614209, ..., -0.02412263,
          0.03870462,  0.01789257],
        [ 0.00946743,  0.00878171, -0.03614209, ..., -0.02412263,
          0.03870462,  0.01789257]]], dtype=float32)>

In [27]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 3.1535041e-02,  6.1979070e-03, -6.7149475e-04,  8.5948110e-03,
        3.2420803e-02, -5.1684864e-03,  4.7606230e-04, -4.3728091e-02,
        3.3342730e-02, -4.8764169e-02, -4.8340335e-03,  3.6802817e-02,
        2.2539590e-02, -4.5949556e-02,  6.5462366e-03, -1.0077547e-02,
       -2.1049604e-03,  3.3208083e-02,  5.1757693e-03,  1.9063130e-03,
       -2.2844851e-02,  5.3234100e-03,  2.7409341e-02, -4.2573810e-03,
        8.4160194e-03, -1.6589761e-03, -6.5664761e-03, -3.5051893e-02,
        3.5674606e-02, -2.6823986e-02,  3.5472307e-02, -4.1850053e-02,
       -3.9753236e-02,  1.7487798e-02,  3.4639541e-02,  4.9679164e-02,
        2.6078437e-02,  3.2774996e-02, -4.6065308e-02,  1.8930163e-02,
       -4.5580506e-02,  3.1200979e-02,  2.5235664e-02, -3.6595024e-02,
        3.7428651e-02,  2.7146351e-02, -2.5210833e-02, -3.4723748e-02,
       -6.2242895e-04, -1.9509424e-02, -2.4154497e-02,  7.9738274e-03,
       -1.6953625e-02, -3.335

## Modelling a text dataset (setting up modelling experiments)

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of the training data

Steps for each model:
1. Construct the model
2. Train the model
3. Make predictions
4. Track the results

### Model 0: Naive Bayes (baseline)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create Tokenization and Modelling Pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline model accuracy: {baseline_score*100:.2f}%")

Baseline model accuracy: 79.27%


In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

### Create an evaluation function for model experiments

Modelling experiments are typically evaluated using:
* Accuracy - the higher the better
* Precision - the higher the better
* Recall - the higher the better
* F1-score - the higher the better

In [31]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.

    Args:
        y_true: true labels in the form of a 1D array
        y_pred: predicted labels in the form of a 1D array

    Returns a dictionary of accuracy, precision, recall, f1-score.
    """

    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100 # get accuracy score in percentage
    # Calculate model precision, recall and f1 score using "weighted average"
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    # Create a dictionary of results
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1 score": model_f1}
    
    return model_results

In [32]:
# Get the baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1 score': 0.7862189758049549}

### Model 1: Feed-forward neural network (dense model)

In [33]:
# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [34]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [35]:
# Build a model using the Sequential API
from tensorflow.keras import layers
input = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(input) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector

output = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary output so use sigmoid activation function
model_1 = tf.keras.Model(input, output, name="model_1_dense") # construct the model

In [36]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
model_1_history = model_1.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20241013-161803
Epoch 1/5
215/215 [==============================] - 3s 6ms/step - loss: 0.6094 - accuracy: 0.6938 - val_loss: 0.5353 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4398 - accuracy: 0.8187 - val_loss: 0.4733 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3455 - accuracy: 0.8603 - val_loss: 0.4565 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2831 - accuracy: 0.8939 - val_loss: 0.4652 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2366 - accuracy: 0.9110 - val_loss: 0.4768 - val_accuracy: 0.7835


In [37]:
# Evaluate the model
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4768 - accuracy: 0.7835


[0.47681620717048645, 0.7834645509719849]

In [38]:
# Make predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.3108622 ],
       [0.71538866],
       [0.9977307 ],
       [0.11487205],
       [0.13306417],
       [0.9354326 ],
       [0.9064813 ],
       [0.9935567 ],
       [0.9685777 ],
       [0.29053026]], dtype=float32)

In [39]:
# Convert model_1 pred probs from probabilities to prediction labels
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [40]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

In [41]:
# Evaluate model_1 with evaluate function
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)

model_1_results

{'accuracy': 78.34645669291339,
 'precision': 0.7884290282979772,
 'recall': 0.7834645669291339,
 'f1 score': 0.7803444000779522}

In [42]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1 score': 0.7862189758049549}

In [43]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

### Visualizing learned embeddings

In [44]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()

len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [46]:
# Get the weight matrix of the embedding layer
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [47]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

### Model 2: LSTM model

* LSTM (Long Short Term Memory) - one of the most popular LSTM cells. LSTM cells have the ability to "remember" things which are important in natural language processing tasks.

In [48]:
# Create LSTM model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")


x = layers.LSTM(64, activation="tanh")(x)
print(f"After LSTM cellL {x.shape}")

outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

After embedding: (None, 15, 128)
After LSTM cellL (None, 64)


In [49]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20241013-161812
Epoch 1/5
215/215 [==============================] - 6s 11ms/step - loss: 0.2177 - accuracy: 0.9229 - val_loss: 0.5982 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.1531 - accuracy: 0.9434 - val_loss: 0.5820 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.1266 - accuracy: 0.9512 - val_loss: 0.7318 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.1052 - accuracy: 0.9583 - val_loss: 0.8029 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0851 - accuracy: 0.9669 - val_loss: 0.8059 - val_accuracy: 0.7690


In [50]:
# Make predictions
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


((762, 1),
 array([[0.03385961],
        [0.8831634 ],
        [0.99982613],
        [0.10482733],
        [0.00123426],
        [0.9989059 ],
        [0.8348501 ],
        [0.9999081 ],
        [0.9997898 ],
        [0.37825468]], dtype=float32))

In [51]:
# Round out pred probs and reduce to 1-dim array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [52]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)

model_2_results

{'accuracy': 76.9028871391076,
 'precision': 0.769926836838627,
 'recall': 0.7690288713910761,
 'f1 score': 0.7673089840953201}

### Model 3: GRU mdoel

* GRU (Gated Recurrent Unit) - a variation of the LSTM cell but with less parameters.

In [53]:
# Build a RNN using the GRU cell
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GRU(64, activation="tanh", return_sequences=True)(x) # return_sequences=True is required for stacking recurrent cells
# print(x.shape)
x = layers.GRU(64, activation="tanh")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_2_GRU")

In [54]:
# Model Summary
model_3.summary()

Model: "model_2_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [55]:
# Compile the model_3
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model_3
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20241013-161825
Epoch 1/5
215/215 [==============================] - 4s 10ms/step - loss: 0.1584 - accuracy: 0.9381 - val_loss: 0.7427 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0830 - accuracy: 0.9691 - val_loss: 0.8390 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0702 - accuracy: 0.9733 - val_loss: 0.8641 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0607 - accuracy: 0.9753 - val_loss: 1.0743 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0504 - accuracy: 0.9774 - val_loss: 1.1759 - val_accuracy: 0.7703


In [56]:
# Make predictions with model_3
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10], model_3_pred_probs.shape

24/24 [==============================] - 0s 3ms/step


(array([[2.0630695e-03],
        [7.2970623e-01],
        [9.9992251e-01],
        [8.2657665e-02],
        [1.1379900e-04],
        [9.9988675e-01],
        [9.3706942e-01],
        [9.9997592e-01],
        [9.9993348e-01],
        [8.1264472e-01]], dtype=float32),
 (762, 1))

In [57]:
# Convert model_3 preds probs into labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [58]:
# Calculate model_3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.03412073490814,
 'precision': 0.7726339681635592,
 'recall': 0.7703412073490814,
 'f1 score': 0.7679241301282296}

### Model 4: Bidirectional-LSTM model

In [59]:
# Build a model with bidrrectional RNN in TensorFlow
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)

x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [60]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [61]:
# Compile the model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20241013-161837
Epoch 1/5
215/215 [==============================] - 6s 14ms/step - loss: 0.1035 - accuracy: 0.9720 - val_loss: 1.0215 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0572 - accuracy: 0.9771 - val_loss: 0.9682 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0467 - accuracy: 0.9793 - val_loss: 1.2574 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0421 - accuracy: 0.9804 - val_loss: 1.4049 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0429 - accuracy: 0.9800 - val_loss: 1.4466 - val_accuracy: 0.7743


In [62]:
# Make predictions with model_4
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10], model_4_pred_probs.shape

24/24 [==============================] - 1s 4ms/step


(array([[5.3438818e-04],
        [8.1377918e-01],
        [9.9998522e-01],
        [4.0919919e-02],
        [2.9281395e-05],
        [9.9993062e-01],
        [9.8144066e-01],
        [9.9999332e-01],
        [9.9998999e-01],
        [9.9888843e-01]], dtype=float32),
 (762, 1))

In [63]:
# Convert pred probs to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [64]:
# Calculate model_4 results
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 77.42782152230971,
 'precision': 0.7744438466099437,
 'recall': 0.7742782152230971,
 'f1 score': 0.7731405762901826}

### Model 5: 1D Convolutional Neural Network

In [84]:
# Test to understand what things look like in Conv1D layer
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sentence into embedding
conv_1d_layer = layers.Conv1D(filters=32,
                              kernel_size=5, # setting this to 5 means it'll look at 5 words at a time, 3 would mean 3 words at a time
                              activation="relu")
conv_1d_output = conv_1d_layer(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
print(f"Embedding output shape: {embedding_test.shape}")
print(f"Conv1D output shape: {conv_1d_output.shape}")
print(f"Max pool output shape: {max_pool_output.shape}")

Embedding output shape: (1, 15, 128)
Conv1D output shape: (1, 11, 32)
Max pool output shape: (1, 32)


In [85]:
# the outputs of each layer
print(f"Embedding output: {embedding_test}")
print(f"Conv1D output: {conv_1d_output}")
print(f"Max pool output: {max_pool_output}")

Embedding output: [[[ 0.00380426 -0.03345213  0.02709604 ... -0.07282466 -0.00750052
   -0.0554304 ]
  [ 0.02370609  0.0435588   0.04986828 ... -0.08142965 -0.03161711
   -0.06529141]
  [-0.06719396  0.04711067  0.03253606 ...  0.0053863  -0.03794164
   -0.00726048]
  ...
  [-0.00919136  0.05643019 -0.02255194 ... -0.04571642  0.01986241
   -0.00391517]
  [-0.00919136  0.05643019 -0.02255194 ... -0.04571642  0.01986241
   -0.00391517]
  [-0.00919136  0.05643019 -0.02255194 ... -0.04571642  0.01986241
   -0.00391517]]]
Conv1D output: [[[0.         0.00734553 0.06702799 0.         0.         0.01305812
   0.03481197 0.07333729 0.08599575 0.01930652 0.         0.01217127
   0.         0.04051074 0.         0.         0.01986402 0.0128359
   0.00645107 0.03505077 0.11143789 0.0417802  0.11257608 0.
   0.         0.06847133 0.09795681 0.         0.         0.
   0.         0.        ]
  [0.         0.         0.07150887 0.06160678 0.0082824  0.
   0.         0.07338902 0.         0.05339599

In [77]:
# Build a model using 1D CNN
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)

x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)  # Reduced kernel size
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid", name="ouput_layer")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_CNN")

In [78]:
model_5.summary()

Model: "model_5_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_6 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_5 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 ouput_layer (Dense)         (None, 1)                 

In [79]:
# Compile the model_5
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model_5
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_5_CNN")])

Saving TensorBoard log files to: model_logs/model_5_CNN/20241013-171557
Epoch 1/5
215/215 [==============================] - 4s 7ms/step - loss: 0.1407 - accuracy: 0.9580 - val_loss: 0.8380 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0758 - accuracy: 0.9720 - val_loss: 0.9893 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0622 - accuracy: 0.9762 - val_loss: 1.0478 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0547 - accuracy: 0.9768 - val_loss: 1.1424 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0509 - accuracy: 0.9772 - val_loss: 1.2700 - val_accuracy: 0.7598


In [80]:
# Make prediction with model_5
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs.shape, model_5_pred_probs[:10]

24/24 [==============================] - 0s 7ms/step


((762, 1),
 array([[9.8299859e-03],
        [8.1947935e-01],
        [9.9974269e-01],
        [4.2985674e-02],
        [9.7570293e-08],
        [9.9517423e-01],
        [9.2876220e-01],
        [9.9985170e-01],
        [9.9999905e-01],
        [7.5481308e-01]], dtype=float32))

In [82]:
# Convert pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [83]:
# Calculate model_5 results
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)

model_5_results

{'accuracy': 75.98425196850394,
 'precision': 0.7633887291688025,
 'recall': 0.7598425196850394,
 'f1 score': 0.7565467702363977}